In [176]:
import csv
import pandas as pd
import re
import nltk
#nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\janva\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [67]:
data = pd.read_csv("../gen/input/gall_product_data_utf16.csv", sep = ";", index_col = 0)

In [68]:
data

,link,type,rating,count,title,country,size,percentage,year,badge,...,main_description,usps,secondary_description,taste,complementaries,grape,label,certificate,producer_description,producer
0,https://www.gall.nl/mucho-mas-tinto-rood-75cl-...,Easy Rider,4.6,200.0,Mucho Más Tinto,Spanje,75CL,14%,NaN,2e halve prijs,...,Kort op hout gerijpte blend van garnacha en te...,"['Bestel voor 22:00 vandaag, morgen in huis', ...","Een wijn gemaakt van druiven uit Toro, La Manc...","specerijen,zachten kers",blokjes kaasen worst,Tempranillo,Bevat sulfur dioxide and sulfits,2 keer bekroond,De familie Solis maakt al sinds 1952 met veel ...,Felix Solis
1,https://www.gall.nl/mucho-mas-blanco-wit-75cl-...,Fruit Lover,4.4,199.0,Mucho Más Blanco,Spanje,75CL,"12,5%",NaN,2e halve prijs,...,Het Spaanse Castilla-La Mancha is het grootste...,"['Bestel voor 22:00 vandaag, morgen in huis', ...",Het doel van deze wijn was om een onderscheide...,fruitigen sappig,paellaen het aperitief,NaN,Bevat sulfur dioxide and sulfits,1 keer bekroond,De familie Solis maakt al sinds 1952 met veel ...,Felix Solis
2,https://www.gall.nl/la-palma-chardonnay-wit-75...,Big White,4.4,200.0,La Palma Chardonnay,Chili,75CL,12%,2022.0,2e halve prijs,...,In deze 100% chardonnay proef je de smaak van ...,"['Bestel voor 22:00 vandaag, morgen in huis', ...",De La Palma wijnen worden gemaakt door het Chi...,"soepel,zachten tropisch fruit",visen kip met roomsaus,Chardonnay,Bevat sulfur dioxide and sulfits,3 keer bekroond,Opgericht in 1824 is Viña la Rosa een van Chil...,Viña La Rosa
3,https://www.gall.nl/ogio-pinot-grigio-wit-75cl...,Fruit Lover,4.1,190.0,Ogio Pinot Grigio,Italië,75CL,12%,2022.0,2e halve prijs,...,Deze Pinot Grigio uit Veneto heeft een heerlij...,"['Bestel voor 22:00 vandaag, morgen in huis', ...",Deze Pinot Grigio uit Veneto heeft een heerlij...,"toegankelijk,meloenen citrus",gegrilde visen pittige gerechten,Pinot grigio,Bevat sulfur dioxide and sulfits,4 keer bekroond,Botter is een familiebedrijf met 70 jaar ervar...,Botter
4,https://www.gall.nl/valdivieso-merlot-rood-75c...,Easy Rider,4.3,199.0,Valdivieso Merlot,Chili,75CL,"13,5%",2021.0,2e halve prijs,...,Bij Valdivieso zorgt de Nieuw-Zeelandse wijnma...,"['Bestel voor 22:00 vandaag, morgen in huis', ...",Bij de wijnen van Valdivieso zorgt de Nieuw-Ze...,"pruimen,bessenen houtrijping",gebraden kipen de borrel,Merlot,Bevat sulfur dioxide and sulfits,3 keer bekroond,Valdivieso is als eerste wijnhuis in Zuid-Amer...,Valdivieso
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1794,https://www.gall.nl/connetable-de-talbot-rood-...,Royal Red,NaN,NaN,Connétable de Talbot,Frankrijk,75CL,"13,5%",2018.0,2e halve prijs,...,De tweede wijn van Château Talbot Grand Cru Cl...,"['Bestel voor 22:00 vandaag, morgen in huis', ...","Tweede wijn van de beroemde Château Talbot, St...",concentratieen verfijning,lamsvleesen gebraden rood vlees,"Cabernet sauvignon, Merlot",Bevat sulfur dioxide and sulfits,1 keer bekroond,Château Talbot is een Quatrième Grand Cru Clas...,Château Talbot
1795,https://www.gall.nl/umani-ronchi-c%C3%BAmaro-c...,Classic Red,NaN,NaN,Umani Ronchi Cúmaro Conero Riserva,Italië,75CL,14%,2016.0,2e halve prijs,...,Houtgerijpte 100% montepulciano uit de Marche ...,"['Bestel voor 22:00 vandaag, morgen in huis', ...",Cúmaro is een belangrijke wijn uit het portfol...,"fris,eleganten mineraal","pasta ,pizzaen vederwild of gerijpte kaas.",Montepulciano,Bevat sulfur dioxide and sulfits,1 keer bekroond,Umani Ronchi is een grote Italiaanse producent...,Umani Ronchi
1796,https://www.gall.nl/penfolds-rwt-barossa-valle...,Big Red,NaN,NaN,Penfolds RWT Barossa Valley Shiraz,Australië,75CL,"14,5%",2016.0,NaN,...,RWT staat voor Red Winemaking Trial. Oorspronk...,"['Bestel voor 22:00 vandaag, morgen in huis', ...",Penfolds RWT Shiraz presenteert zich als een w...,NaN,NaN,Shiraz,1 keer bekroond,NaN,Vraag een willekeurige wijnliefhebber naar de ...,Penfolds Barossa Valley
1797,https://www.gall.nl/ch%C3%A2

In [69]:
#Remove percent sign from alcohol column
data.percentage = data.percentage.str.replace('%','').str.replace(',','.').astype(float)

In [70]:
#Cleaning the size column
##Removing 'cl'
without_cl = [str(i).replace('CL', '') for i in data['size']]
cleaned_cl = []
for size in without_cl:
    size = size.replace(',', '.')
    ##If there are multiple bottles, multiply the sizes
    if 'X' in size:
        size = float(size.split("X")[0]) * float(size.split("X")[1])
    cleaned_cl.append(size)
data['cleaned_size'] = cleaned_cl

In [71]:
varieties = []
for variety in data['grape']:
    for i in str(variety).split(','):
        varieties.append(i.strip())
set(varieties)

{'Agiorgitiko',
 'Aglianico',
 'Airén',
 'Albariño',
 'Alicante bouschet',
 'Alvarinho',
 'Arinto',
 'Arneis',
 'Auxerrois',
 'Avesso',
 'Baga',
 'Barbera',
 'Blauer zweigelt',
 'Blaufränkisch',
 'Bobal',
 'Bonarda',
 'Brachetto',
 'Cabernet franc',
 'Cabernet sauvignon',
 'Cannonau',
 'Carignan',
 'Carignano',
 'Carmenère',
 'Carricante',
 'Chardonnay',
 'Chenin blanc',
 'Cinsault',
 'Clairette',
 'Cortese',
 'Corvina',
 'Dolcetto',
 'Dornfelder',
 'Falanghina',
 'Fiano',
 'Frappato',
 'Friulano',
 'Furmint',
 'Gamay',
 'Garganega',
 'Garnacha',
 'Garnacha blanca',
 'Gewürztraminer',
 'Glera',
 'Godello',
 'Grechetto',
 'Greco',
 'Grenache',
 'Grenache blanc',
 'Grenache noir',
 'Grillo',
 'Gros manseng',
 'Grüner veltliner',
 'Lagrein',
 'Lambrusco',
 'Macabeo',
 'Malbec',
 'Malvasia',
 'Marsanne',
 'Mazuelo',
 'Melon de Bourgogne',
 'Merlot',
 'Molinara',
 'Monastrell',
 'Montepulciano',
 "Montepulciano d'Abruzzo",
 'Moscato',
 'Mourvèdre',
 'Muller thurgau',
 'Muscadelle',
 'Muscat

In [72]:
for column in varieties:
    data[column] = 0

In [58]:
for index, row in data.iterrows():
    if ',' in row['grape']:
    all_variety = row['grape'].split(',')
    for variety in all_variety:
        if variety in varieties:
            row[variety] = 1

Tempranillo
nan
Chardonnay
Pinot grigio
Merlot
Malbec, Syrah, Bonarda
Sauvignon blanc
Merlot, Cabernet sauvignon
Merlot
Merlot, Syrah
Sauvignon blanc
Primitivo, Montepulciano d'Abruzzo, Nero d'avola, Merlot
Chardonnay
Carmenère
Chardonnay
Chardonnay
Merlot
Chardonnay
Sauvignon blanc
Chardonnay
Cabernet sauvignon
Cabernet sauvignon
Sauvignon blanc
Chardonnay
Sauvignon blanc
Sauvignon blanc
Malbec, Syrah
Corvina, Molinara, Rondinella
Tempranillo
Chardonnay
Cabernet sauvignon
Sauvignon blanc
nan
Shiraz
Viognier, Chardonnay
Cabernet sauvignon
Tempranillo, Garnacha
Chardonnay
Cabernet sauvignon
Chardonnay
Malbec
Sauvignon blanc
Sauvignon blanc
Pinot grigio
Tempranillo, Mazuelo
Chardonnay, Viognier
Garnacha
Pinot noir, Syrah, Cabernet franc, Cabernet sauvignon
Macabeo
Garnacha
nan
Grenache, Mourvèdre, Cinsault, Syrah
Sauvignon blanc
Syrah, Grenache
Garnacha
Chenin blanc
Shiraz
Sauvignon blanc
Chardonnay
Malbec, Pinotage
Syrah, Grenache
Chardonnay, Viognier
Sauvignon blanc
nan
Riesling
Chardo

Cabernet sauvignon, Merlot, Petit verdot
Grenache, Syrah
Carignan
Grenache, Cinsault, Syrah, Mourvèdre
Corvina, Rondinella, Molinara
Cabernet franc
nan
Grenache, Syrah, Mourvèdre
Grenache, Syrah
Verdejo
Ribolla gialla
Cabernet sauvignon
Malbec
Tempranillo, Petit verdot
Nero d'avola
Chardonnay
Syrah, Petit verdot, Cabernet franc
Sangiovese, Merlot
Carricante
Syrah
Viura
Chardonnay
Sauvignon blanc
Shiraz
Sangiovese
Barbera, Pinot noir, Nebbiolo, Dolcetto
Vermentino
Pinot noir
Pinot noir
Syrah, Grenache
Moscato
Cabernet franc, Merlot
Sangiovese, Merlot
Nebbiolo
Vermentino
Viognier
Cabernet franc
Chardonnay
Pinot noir, Merlot, Dornfelder
Frappato
nan
Grillo, Sauvignon blanc
nan
Brachetto
Falanghina
Roussanne, Clairette, White grenache
Garnacha
Chardonnay
Chardonnay
Furmint
Refosco
Pinot noir
Chardonnay
Sangiovese
Barbera, Merlot
Chardonnay
Chardonnay
Pinot noir
Chardonnay
nan
Syrah, Viognier
nan
Syrah, Grenache
Tempranillo
Tempranillo
nan
Mazuelo, Tempranillo
Pinot noir
Lagrein
Pinot noir


Pinot blanc
Gamay
Sauvignon blanc
Sauvignon blanc
Merlot
Cabernet franc
Weissburgunder
Malbec
nan
nan
Schioppettino
Merlot, Cabernet sauvignon, Cabernet franc
Tempranillo
Ugni blanc
nan
Malbec
nan
nan
Syrah, Grenache
Merlot, Cabernet sauvignon
Cinsault, Cabernet sauvignon, Syrah, Viognier, Grenache
nan
Chardonnay
Zinfandel
Gamay
Cabernet sauvignon, Merlot
Zinfandel
Cabernet sauvignon
Tempranillo
Spätburgunder
Arneis
Merlot
Chardonnay
Cabernet sauvignon, Cabernet franc, Merlot, Petit verdot
Vermentino
Chardonnay
Gewürztraminer
Nebbiolo
Riesling
Riesling
Merlot, Cabernet sauvignon, Petit verdot, Cabernet franc
Pinot noir
Gamay
Chardonnay
Cannonau, Carignan
Pinotage
Cabernet sauvignon, Merlot, Cabernet franc
Nero d'avola
Marsanne
Cabernet sauvignon, Merlot, Petit verdot, Cabernet franc
Chardonnay
Chardonnay
Cabernet sauvignon, Merlot, Cabernet franc
Zinfandel
Rondinella, Corvina, Molinara
nan
Cabernet sauvignon, Merlot, Cabernet franc, Petit verdot
Sauvignon blanc
Malbec
Nebbiolo
Temprani

In [75]:
for col in data.iloc[:,22:].columns:
    # check if the column name exists in the value of another column
    data.loc[data['grape'].str.contains(col, na = False), col] = 1

In [79]:
data.iloc[:,22:]

,Tempranillo,nan,Chardonnay,Pinot grigio,Merlot,Malbec,Syrah,Bonarda,Sauvignon blanc,Cabernet sauvignon,...,Sangiovese grosso,Muscat de Frontignan,Carignano,Trebbiano di Lugana,Garnacha blanca,Muscadelle,Arinto,Schioppettino,Palomino,Bobal
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1794,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1795,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1796,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1797,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
awards = pd.read_csv("../gen/input/gall_awards.csv", sep = ";")
awards

,product_link,award_link,vintage,grade,contributor
0,https://www.gall.nl/mucho-mas-tinto-rood-75cl-...,https://www.gall.nl/on/demandware.store/Sites-...,Oogstjaar 0000,90 punten,Gilbert and Gaillard
1,https://www.gall.nl/mucho-mas-tinto-rood-75cl-...,https://www.gall.nl/on/demandware.store/Sites-...,NaN,7+,De Grote Hamersma
2,https://www.gall.nl/mucho-mas-blanco-wit-75cl-...,https://www.gall.nl/on/demandware.store/Sites-...,Oogstjaar,8,De Grote Hamersma
3,https://www.gall.nl/la-palma-chardonnay-wit-75...,https://www.gall.nl/on/demandware.store/Sites-...,Oogstjaar 2020,8.5,De Grote Hamersma
4,https://www.gall.nl/la-palma-chardonnay-wit-75...,https://www.gall.nl/on/demandware.store/Sites-...,Oogstjaar 2021,8.5,De Grote Hamersma
...,...,...,...,...,...
4965,https://www.gall.nl/ch%C3%A2teau-barde-haut-ro...,https://www.gall.nl/on/demandware.store/Sites-...,Oogstjaar 2016,p_94 punten,Wine Advocate
4966,https://www.gall.nl/ch%C3%A2teau-barde-haut-ro...,https://www.gall.nl/on/demandware.store/Sites-...,NaN,p_95 punten,Vinous
4967,https://www.gall.nl/ch%C3%A2teau-haut-brion-ro...,https://www.gall.nl/on/demandware.store/Sites-...,Oogstjaar 2012,93 punten,Decanter
4968,https://www.gall.nl/ch%C3%A2teau-haut-brion-ro...,https://www.gall.nl/on/demandware.store/Sites-...,NaN,98 punten,Wine Advocate


In [90]:
awards.vintage.fillna(method = "ffill", inplace = True)

In [92]:
awards.vintage = awards.vintage.str.replace('Oogstjaar ', '')

In [95]:
DGH_awards = awards[awards.contributor == "De Grote Hamersma"]

In [133]:
DGH_awards['grade_clean'] = DGH_awards.grade.str.replace('+', '.25')

<ipython-input-133-f89d961e0f33>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DGH_awards['grade_clean'] = DGH_awards.grade.str.replace('+', '.25')


In [140]:
cleaned_grades = []
for grade in DGH_awards['grade_clean']:
    if '-' in grade:
        grade = grade.replace('-', '.75')
        grade = re.findall(r"[-+]?(?:\d*\.*\d+)", str(grade).strip())
        if len(grade) >0:
            grade = float(grade[0])-1
        else:
            grade = None
    else:
        grade = re.findall(r"[-+]?(?:\d*\.*\d+)", str(grade).strip())[0]
    cleaned_grades.append(float(grade))

In [141]:
DGH_awards['grade_clean'] = cleaned_grades

<ipython-input-141-544445389c1a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DGH_awards['grade_clean'] = cleaned_grades


In [142]:
cleaned_grades

[7.25,
 8.0,
 8.5,
 8.5,
 8.5,
 7.0,
 7.25,
 7.0,
 7.0,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.5,
 8.75,
 9.0,
 8.75,
 8.25,
 8.25,
 8.5,
 8.0,
 8.5,
 8.5,
 7.5,
 8.0,
 8.25,
 8.75,
 8.75,
 8.0,
 7.5,
 7.5,
 7.5,
 8.0,
 8.0,
 7.0,
 8.0,
 8.5,
 8.5,
 8.5,
 9.0,
 8.0,
 7.75,
 7.75,
 8.0,
 8.0,
 8.5,
 8.5,
 8.0,
 8.0,
 8.5,
 8.0,
 8.5,
 8.25,
 8.5,
 9.0,
 7.5,
 7.5,
 8.0,
 7.5,
 8.0,
 8.25,
 8.5,
 8.5,
 8.5,
 7.5,
 8.25,
 8.5,
 8.5,
 7.5,
 7.5,
 7.5,
 7.5,
 8.0,
 8.0,
 8.5,
 7.5,
 8.0,
 8.5,
 9.0,
 8.25,
 8.5,
 7.0,
 8.25,
 8.25,
 8.0,
 7.0,
 7.0,
 7.0,
 8.0,
 8.5,
 7.0,
 8.0,
 8.0,
 8.0,
 8.75,
 8.25,
 7.5,
 7.75,
 8.5,
 8.25,
 7.0,
 8.5,
 8.25,
 8.25,
 7.5,
 7.5,
 8.5,
 9.0,
 9.0,
 7.75,
 8.25,
 7.5,
 8.0,
 8.25,
 8.5,
 8.75,
 7.0,
 7.75,
 8.5,
 8.0,
 8.5,
 8.5,
 8.5,
 8.0,
 8.25,
 8.5,
 8.0,
 7.5,
 8.25,
 8.5,
 8.0,
 8.25,
 8.5,
 8.5,
 7.5,
 7.5,
 8.0,
 8.5,
 8.5,
 8.5,
 8.5,
 -0.241,
 8.5,
 7.5,
 8.5,
 8.5,
 8.25,
 7.75,
 8.0,
 8.25,
 8.5,
 8.5,
 8.5,
 8.0,
 8.0,
 8.5,
 8.0,
 8.0,
 8.0,
 8.

In [144]:
link_grade = DGH_awards.groupby('product_link')['grade_clean'].mean()

In [147]:
data = pd.merge(data,link_grade, left_on = "link", right_on = "product_link", how = "left")

In [ ]:
#Subtlex-nl import

In [151]:
subtlex = pd.read_excel("../data/SUBTLEX-NL.xlsx")
subtlex

,Word,FREQcount,CDcount,FREQlow,CDlow,FREQlemma,SUBTLEXWF,Zipf,SUBTLEXCD,Lg10CD,dominant.pos,dominant.pos.freq,dominant.pos.lemma,dominant.pos.lemma.freq,all.pos,all.pos.freq,all.pos.lemma.freq
0,ik,1744062,8054,778704,3125,1744527,39883.0334,7.597064,99.8017,3.9061,VNW,1743609,ik,1743944,.VNW.SPEC.N.VZ.,.1743609.448.4.1.,.1743944.448.134.1.
1,je,1600888,8060,1315051,6535,1600923,36608.9449,7.559864,99.8761,3.9064,VNW,1600798,je,1600798,.VNW.SPEC.N.BW.LID.,.1600798.72.15.2.1.,.1600798.72.50.2.1.
2,het,1068396,8066,780771,5578,1913811,24431.9717,7.384235,99.9504,3.9067,VNW,735390,het,735395,.VNW.LID.SPEC.WW.N.,.735390.332929.53.22.2.,.735395.332929.53.845403.31.
3,de,1061177,8070,903872,6512,1063827,24266.8883,7.381291,100.0000,3.9069,LID,1060098,de,1062748,.LID.VNW.SPEC.VZ.,.1060098.806.272.1.,.1062748.806.272.1.
4,dat,965424,8063,715570,6107,965431,22077.2184,7.340221,99.9133,3.9066,VNW,532576,dat,532576,.VNW.VG.SPEC.N.WW.,.532576.432794.51.2.1.,.532576.432794.51.9.1.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150352,'24,2,2,2,2,2,0.0457,1.832624,0.0248,0.4771,TW,2,'24,2,.TW.,.2.,.2.
150353,'09,2,2,2,2,2,0.0457,1.832624,0.0248,0.4771,TW,2,'09,2,.TW.,.2.,.2.
150354,'02,2,2,2,2,2,0.0457,1.832624,0.0248,0.4771,TW,2,'02,2,.TW.,.2.,.2.
150355,&_fire,2,2,2,2,2,0.0457,1.832624,0.0248,0.4771,SPEC,2,&_fire,2,.SPEC.,.2.,.2.


In [170]:
top_3000 = list(subtlex.loc[:3000,'Word'])

In [171]:
top_3000_lemma = list(subtlex.loc[:3000,'dominant.pos.lemma'])

In [180]:
scores = []
for desc in data['main_description']:
    familiarity = 0
    bag_of_words = desc.split()
    for word in bag_of_words:
        if word.lower() in top_3000:
            familiarity += 1
        elif word.lower() in top_3000_lemma:
            familiarity += 1
    unfamiliarity = 1- familiarity / len(bag_of_words)
    sentence_len = len(bag_of_words) / len(sent_tokenize(desc))
    score = 0.0496 * sentence_len + 0.1579 * unfamiliarity + 3.6365
    scores.append(score)

In [182]:
data['NDC_scores'] = scores

In [188]:
data.to_csv('gall_clean.csv', sep = ";")